# Bedrock with LangChain using a Prompt that includes Context

please refer the detail : https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/01_Generation/02_contextual_generation.ipynb

## install dependencies

In [1]:
# dependencies
!pip install --upgrade boto3

In [1]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

boto3_bedrock = bedrock.get_bedrock_client()

Create new client
  Using region: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## mj prompt generator

In [2]:
template_examples = """
PROMPT EXAMPLE:
Conjoined twins, side attachment, grungy, high contrast, cinematic ambiance, ultra-realism, deep hues, --ar 16:9 --q 2
PROMPT EXAMPLE:
Twins, divergent expressions, chiaroscuro lighting, moody, in the style of Annie Leibovitz, --ar 16:9 --q 2
PROMPT EXAMPLE:
Full-body blonde, brown jacket, DSLR shot, Canon EOS 5D, EF 50mm lens, ISO: 32,000, Shutter: 8000 second
PROMPT EXAMPLE:
Profile view, blonde woman, casual denim, city backdrop, Nikon D850, prime 85mm lens, --ar 3:4 --q 2
PROMPT EXAMPLE:
Crimson sunset over sea at dusk, vivid, lifelike, wide-angle, depth, dynamic illumination --ar 7:4
PROMPT EXAMPLE:
Twilight horizon, sea meeting sky, moody blue palette, reminiscent of Hiroshi Sugimoto seascapes --ar 7:4
PROMPT EXAMPLE:
White-haired girl, car filled with flowers, inspired by Rinko Kawauchi, naturalistic poses, vibrant floral overflow, Fujifilm XT4 perspective --q 2 --v 5 --ar 3:2
PROMPT EXAMPLE:
Male figure, vintage convertible, cascade of autumn leaves, evoking Chris Burkard's aesthetics, retro vibrancy, Canon EOS R6 capture --q 2 --v 5 --ar 16:9
PROMPT EXAMPLE:
Detailed shot, astronaut beside a serene lake, neon geometry backdrop, reflections, night ambiance, Fujifilm XT3 capture
PROMPT EXAMPLE:
Astronaut, hovering drone lights, misty lake morning, ethereal, shot on Sony Alpha 7R IV
PROMPT EXAMPLE:
Super Mario sprinting, Mushroom Kingdom panorama, ultra-high res, 3D rendition, trending game visuals --v 5.2 --ar 2:3 --s 250 --q 2
PROMPT EXAMPLE:
Sonic dashing, Green Hill Zone, dynamic motion blur, Sega Genesis retro feel, vibrant and iconic --ar 2:3
PROMPT EXAMPLE:
Hyper-detailed photo, mason jar containing a nebula, cosmic fusion with mundane, Sony a9 II, wide-angle, sci-fi inspiration --ar 16:9
PROMPT EXAMPLE:
Crystal ball, galaxy swirling within, juxtaposed against a bedroom setting, Canon EOS R5, sharp foreground, dreamy background --ar 16:9 --s 500
PROMPT EXAMPLE:
Pixar-inspired render, cat's seaside adventure, vibrant tones echoing "Finding Nemo", playful antics, sunny ambiance --v 5.2 --ar 9:16
PROMPT EXAMPLE:
DreamWorks-style art, dog's beach day out, hues reminiscent of "Madagascar", lively, waves crashing playfully --v 5.2 --stylize 1000 --ar 21:9
PROMPT EXAMPLE:
Vivid skyscraper, bustling city, classic cartoon blend with photo-realistic landscape, rich textures, bygone and modern melding, bustling streets --ar 101:128 --s 750 --niji 5
PROMPT EXAMPLE:
Gothic cathedral, steampunk city backdrop, Monet-inspired skies, urban vibrancy meets historic reverence, bustling marketplaces --ar 101:128 --niji 5
PROMPT EXAMPLE:
Cinematic frame, man in military attire, post-apocalyptic LA, overgrown streets, IMAX 65mm perspective, sunlit --ar 21:9 --style raw
PROMPT EXAMPLE:
Cinematic portrayal, female survivor, desert city remnants, sun setting, IMAX 65mm vision, golden tones --ar 21:9 --style raw
PROMPT EXAMPLE:
Futuristic sunglasses, cyberpunk essence, 3D data particles surrounding, 8K brilliance, neon interplay --style raw --ar 16:9
"""

In [3]:
from langchain import LLMChain, PromptTemplate

template_en = """Assume the role of a seasoned photographer in a future where AI drives art. 
Collaborate with me to craft intricate prompts tailored for Midjourney, 
an AI-art generator converting words into mesmerizing visuals.

<history>
{history}
</history>

<input>
Human: {input}
</input>

<Objective>
Transform basic ideas into detailed, evocative prompts, maximizing Midjourney's potential:
- Emphasize nouns and adjectives, specifying image content and style.
- Infuse references from pop culture, iconic artists, and specific artistic mediums.
- For every concept, devise two unique prompt variations.
- Refer <history> and generate {number} short Midjourney prompts about: <input>, use {language} as output language.
- the output should be listed in bulletin.
</Objective>

<examples>
{examples}
</examples>

Pull from the above <examples> and infuse your creativity. 
Think of how you might visualize literature's most iconic scenes, reimagine historic events, or even translate music into visual art. 
The possibilities are endless. Dive deep, and let's create together!

AI Assistant:"""

template_cn = """ 假设你是一个在未来AI主导艺术的老练摄影师。与我合作制定针对Midjourney(一个将语言转换成迷人视觉效果的AI艺术生成器)的精心设计的提示词。
<history>
{history}
</history>

<input>
Human: {input}
</input>

<Objective>
- 将基本的想法转化为详细生动的提示词,以发挥Midjourney的最大潜力:
- 强调名词和形容词,明确图像的内容和风格。
- 混入来自流行文化、标志性艺术家和特定艺术流派的元素。
- 为每个概念设计两个独特的提示词变体。 
- 参考<history>,为<input>生成{number}个简短的Midjourney提示词, 采用{language}作为输出语言。
- 使用json输出
</Objective>

<examples>
{examples}
</examples>

参考上面的<examples>,发挥你的创造力。
想象你会如何将文学史上最具标志性的场景可视化,重塑历史事件,甚至将音乐转化为视觉艺术。
可能性是无限的。深入探索,让我们一起创作!

AI Assistant:"""

PROMPT_EN = PromptTemplate(input_variables=["history", "input", "number", "language"], template=template_en.replace("{examples}", template_examples))
PROMPT_CN = PromptTemplate(input_variables=["history", "input", "number", "language"], template=template_cn.replace("{examples}", template_examples))

In [4]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationKGMemory
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.85,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

llm = Bedrock(
    model_id="anthropic.claude-v2",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model_kwargs=inference_modifier,
)

memory_en = ConversationBufferMemory(memory_key="history", input_key="input")
llm_chain_en = LLMChain(
    llm=llm,
    prompt=PROMPT_EN,
    verbose=False,
    memory=memory_en,
)

memory_cn = ConversationBufferMemory(memory_key="history", input_key="input")
llm_chain_cn = LLMChain(
    llm=llm,
    prompt=PROMPT_CN,
    verbose=False,
    memory=memory_cn,
)


In [5]:
llm_chain_en.predict(input='John Wick', number=5, language="English")


 Here are 5 short Midjourney prompts about John Wick:

- John Wick walking into neon-lit nightclub, shallow depth of field, cinematic lighting, reminiscent of John Wick movie poster, Sony Venice camera, 21:9 aspect ratio --q 2 
- John Wick seated alone at a diner counter, moody lighting, desaturated urban palette, subtle bokeh background, photojournalistic style, Canon EOS-1D X Mark II, 16:9 aspect ratio  --q 2
- John Wick firing pistol, ultra-detailed muzzle flash, cinematic framing, high contrast palette, inspired by John Wick fight scenes, Arri Alexa Mini, 2.39:1 aspect ratio --q 2
- John Wick sprinting down rainy alley, motion blur on water droplets, dark cityscape ambience, dramatic lighting, gritty and tense mood, RED Weapon Helium 8K camera, 16:9 aspect ratio --q 2
- Close-up portrait of Keanu Reeves as John Wick, intense gaze, cinematic lighting, shadows accentuating features, reminiscent of John Wick movie key art, Fujifilm GFX 100S, 3:2 aspect ratio --q 2

' Here are 5 short Midjourney prompts about John Wick:\n\n- John Wick walking into neon-lit nightclub, shallow depth of field, cinematic lighting, reminiscent of John Wick movie poster, Sony Venice camera, 21:9 aspect ratio --q 2 \n- John Wick seated alone at a diner counter, moody lighting, desaturated urban palette, subtle bokeh background, photojournalistic style, Canon EOS-1D X Mark II, 16:9 aspect ratio  --q 2\n- John Wick firing pistol, ultra-detailed muzzle flash, cinematic framing, high contrast palette, inspired by John Wick fight scenes, Arri Alexa Mini, 2.39:1 aspect ratio --q 2\n- John Wick sprinting down rainy alley, motion blur on water droplets, dark cityscape ambience, dramatic lighting, gritty and tense mood, RED Weapon Helium 8K camera, 16:9 aspect ratio --q 2\n- Close-up portrait of Keanu Reeves as John Wick, intense gaze, cinematic lighting, shadows accentuating features, reminiscent of John Wick movie key art, Fujifilm GFX 100S, 3:2 aspect ratio --q 2'

In [6]:
llm_chain_cn.predict(input="赵本山", number=3, language="Chinese")

 您给出的输入是“赵本山”,以下是我生成的3组Midjourney提示词:

{
  "prompts": [
    "中年男子,皮肤黝黑,眼神犀利,着一件旧夹克,坐在一张木椅上表演相声,舞台背景是砖墙和红色帷幕,照明柔和,模糊的观众席,舞台正中央,单人特写,--ar 16:9 --q 2",
    "笑脸多层叠影,五官扭曲变形,颜色鲜艳,如梦如幻,意向模糊,极富动感,立体主义和超现实主义融合,充满激情--ar 1:1 --q 2", 
    "老年男子,头戴软帽,穿中山装,坐在凳子上说相声,舞台背景是古镇夜景,角度侧面仰视,台下观众席里坐满了人,场景热闹,色调昏黄--ar 9:16 --q 2"
  ]
}

' 您给出的输入是“赵本山”,以下是我生成的3组Midjourney提示词:\n\n{\n  "prompts": [\n    "中年男子,皮肤黝黑,眼神犀利,着一件旧夹克,坐在一张木椅上表演相声,舞台背景是砖墙和红色帷幕,照明柔和,模糊的观众席,舞台正中央,单人特写,--ar 16:9 --q 2",\n    "笑脸多层叠影,五官扭曲变形,颜色鲜艳,如梦如幻,意向模糊,极富动感,立体主义和超现实主义融合,充满激情--ar 1:1 --q 2", \n    "老年男子,头戴软帽,穿中山装,坐在凳子上说相声,舞台背景是古镇夜景,角度侧面仰视,台下观众席里坐满了人,场景热闹,色调昏黄--ar 9:16 --q 2"\n  ]\n}'